In [35]:

#CONNECT TO SPOTIFY

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="a09d11aa9cb341c899d56d84c67f5d7f" 
secret = "fa09d00c868d497ebbb160476b56f097"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [36]:
#GET SONG METRICS FROM PLAYLIST

#connect to playlist
playlist = sp.user_playlist("userid", "6ZqQYzz5AzD8nwL5WYlKBS") 
songs = playlist["tracks"]["items"] 
ids = [] 

#obtain data
for i in range(len(songs)): 
    ids.append(songs[i]["track"]["id"]) 
features = sp.audio_features(ids) 
dfmet = pd.DataFrame(features)

dfmet.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 18 columns):
acousticness        19 non-null float64
analysis_url        19 non-null object
danceability        19 non-null float64
duration_ms         19 non-null int64
energy              19 non-null float64
id                  19 non-null object
instrumentalness    19 non-null float64
key                 19 non-null int64
liveness            19 non-null float64
loudness            19 non-null float64
mode                19 non-null int64
speechiness         19 non-null float64
tempo               19 non-null float64
time_signature      19 non-null int64
track_href          19 non-null object
type                19 non-null object
uri                 19 non-null object
valence             19 non-null float64
dtypes: float64(9), int64(4), object(5)
memory usage: 2.8+ KB


In [53]:
#SONG METRICS DATA CLEANUP
columns_to_drop_met = ['analysis_url','track_href','type','uri']
dfmet.drop(columns_to_drop_met, axis=1,inplace=True)

dfmet.rename(columns={'id': 'track_id'}, inplace=True)

dfmet.shape
dfmet.info()
dfmet.head()

#Song metrics collection is done here

KeyError: "['analysis_url' 'track_href' 'type' 'uri'] not found in axis"

In [37]:
#GET SONG NAME FROM PLAYLIST


# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []


#Get playlist
track_results = sp.user_playlist("userid", "6ZqQYzz5AzD8nwL5WYlKBS")["tracks"]["items"]


for t in track_results:
    artist_name.append(t['track']['artists'][0]['name'])
    track_name.append(t['track']['name'])
    track_id.append(t['track']['id'])
    popularity.append(t['track']['popularity'])
print('number of elements in the track_id list:', len(track_id))

#imports track names
import pandas as pd
dftracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)

dftracks.shape
dftracks.info()
dftracks.head()

number of elements in the track_id list: 19
(100, 4)


,artist_name,track_name,track_id,popularity
0,J Balvin,Mi Gente,4ipnJyDU3Lq15qBAYNqlqK,71
1,Rae Sremmurd,Powerglide (feat. Juicy J) - From SR3MM,1BuZAIO8WZpavWVbbq3Lci,73
2,Justin Timberlake,Say Something,1LhMopPAallLeaeNutqbgS,72
3,6ix9ine,GUMMO,4HbiCvH1R7mVOJ7KY7JQBD,68
4,Cardi B,Bodak Yellow,2771LMNxwf62FTAdpJMQfM,67


In [49]:
#SONG NAME DATA CLEANUP

grouped = dftracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

dftracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

# doing the same grouping as before to verify the solution
grouped_after_dropping = dftracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

dftracks[dftracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()
dftracks.shape

(19, 4)

In [54]:
# MERGE BOTH DATAFRAMES

# the 'inner' method will make sure to keep track IDs present in both datasets
dftotal = pd.merge(dftracks,dfmet,on='track_id',how='inner')
dftotal.head()
dftotal.info()


dftotal[dftotal.duplicated(subset=['artist_name','track_name'],keep=False)]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 17 columns):
artist_name         19 non-null object
track_name          19 non-null object
track_id            19 non-null object
popularity          19 non-null int64
acousticness        19 non-null float64
danceability        19 non-null float64
duration_ms         19 non-null int64
energy              19 non-null float64
instrumentalness    19 non-null float64
key                 19 non-null int64
liveness            19 non-null float64
loudness            19 non-null float64
mode                19 non-null int64
speechiness         19 non-null float64
tempo               19 non-null float64
time_signature      19 non-null int64
valence             19 non-null float64
dtypes: float64(9), int64(5), object(3)
memory usage: 2.7+ KB


,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [ ]:
#write to csv
dftotal.to_csv('Spotifyscrape.csv')

In [55]:
dftotal

,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,J Balvin,Mi Gente,4ipnJyDU3Lq15qBAYNqlqK,71,0.017900,0.774,189440,0.687,0.000012,11,0.1280,-4.818,0,0.0584,104.959,4,0.3080
1,Rae Sremmurd,Powerglide (feat. Juicy J) - From SR3MM,1BuZAIO8WZpavWVbbq3Lci,73,0.016800,0.713,332301,0.831,0.000000,1,0.1180,-4.750,0,0.1500,173.948,4,0.5840
2,Justin Timberlake,Say Something,1LhMopPAallLeaeNutqbgS,72,0.103000,0.707,278893,0.632,0.000011,10,0.0841,-7.031,1,0.0789,97.040,4,0.3720
3,6ix9ine,GUMMO,4HbiCvH1R7mVOJ7KY7JQBD,68,0.055300,0.660,157643,0.775,0.000000,2,0.1290,-4.926,1,0.1720,157.036,4,0.6350
4,Cardi B,Bodak Yellow,2771LMNxwf62FTAdpJMQfM,67,0.067200,0.929,223962,0.723,0.000000,11,0.3460,-5.792,0,0.1090,125.022,4,0.4580
5,Childish Gambino,This Is America,0b9oOr2ZgvyQu88wzixux9,74,0.117000,0.854,225773,0.463,0.000000,5,0.3540,-6.159,1,0.1370,120.024,4,0.5490
6,N.E.R.D,Lemon,4PpuH4mxL0rD35mOWaLoKS,64,0.002140,0.916,219707,0.727,0.000145,1,0.1180,-6.556,1,0.0861,94.997,4,0.2010
7,Jay Rock,"King's Dead (with Kendrick Lamar, Future & Jam...",51rXHuKN8Loc4sUlKPODgH,72,0.000874,0.563,229670,0.717,0.000000,1,0.1830,-5.186,1,0.3370,82.064,4,0.4270
8,Thomas Rhett,Marry Me,0OWZFobGSIW9GrSlQ9C5pc,70,0.113000,0.588,205453,0.408,0.000000,3,0.1300,-8.662,1,0.0237,85.008,4,0.4840
9,Miguel,Sky Walker (feat. Travis Scott),5WoaF1B5XIEnWfmb5NZikf,72,0.173000,0.674,259333,0.506,0.000000,6,0.0866,-10.532,1,0.0434,145.965,4,0.1830
